In [80]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_core.charts import format_fig

In [81]:

reform = Reform.from_dict({
  "gov.aca.ptc_phase_out_rate[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.ptc_phase_out_rate[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.ptc_phase_out_rate[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.ptc_phase_out_rate[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")



In [82]:

situation_vermont = {
  "people": {
    "you": {
      "age": {
        "2026": 30
      }
    },
    "your partner": {
      "age": {
        "2026": 30
      }
    },
    "your first dependent": {
      "age": {
        "2026": 3
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ],
      "state_name": {
        "2026": "NY"
      }
    }
  },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    },
    "your first dependent's marital unit": {
      "members": [
        "your first dependent"
      ],
      "marital_unit_id": {
        "2026": 1
      }
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 800,
        "min": 0,
        "max": 400000
      }
    ]
  ]
}



In [83]:
situation_texas = {
  "people": {
    "you": {
      "age": {
        "2026": 25
      }
    },
    "your partner": {
      "age": {
        "2026": 28
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ],
      "state_name": {
        "2026": "TX"
      },
      "county_fips": {
        "2026": "48015"
      }
    }
  },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 800,
        "min": 0,
        "max": 400000
      }
    ]
  ]
}



In [84]:
simulation_vermont = Simulation(
    situation=situation_vermont,
)
simulation_texas = Simulation(
    situation=situation_texas,
)
reformed_simulation_vermont = Simulation(
    situation=situation_vermont,
    reform=reform,
)
reformed_simulation_texas = Simulation(
    situation=situation_texas,
    reform=reform,
)

In [85]:
household_income_vermont = simulation_vermont.calculate("employment_income", map_to="household", period=2026)
baseline_vermont_per_capita_chip = simulation_vermont.calculate("per_capita_chip", map_to="household", period=2026)
baseline_vermont_aca_ptc = simulation_vermont.calculate("aca_ptc", map_to="household", period=2026)
baseline_vermont_medicaid_cost = simulation_vermont.calculate("medicaid_per_capita_cost", map_to="household", period=2026)

reform_vermont_per_capita_chip = reformed_simulation_vermont.calculate("per_capita_chip", map_to="household", period=2026)
reform_vermont_aca_ptc = reformed_simulation_vermont.calculate("aca_ptc", map_to="household", period=2026)
reform_vermont_medicaid_cost = reformed_simulation_vermont.calculate("medicaid_per_capita_cost", map_to="household", period=2026)

# Get household-level values for Texas
household_income_texas = simulation_texas.calculate("employment_income", map_to="household", period=2026)
baseline_texas_per_capita_chip = simulation_texas.calculate("per_capita_chip", map_to="household", period=2026)
baseline_texas_aca_ptc = simulation_texas.calculate("aca_ptc", map_to="household", period=2026)
baseline_texas_medicaid_cost = simulation_texas.calculate("medicaid_per_capita_cost", map_to="household", period=2026)

reform_texas_per_capita_chip = reformed_simulation_texas.calculate("per_capita_chip", map_to="household", period=2026)
reform_texas_aca_ptc = reformed_simulation_texas.calculate("aca_ptc", map_to="household", period=2026)
reform_texas_medicaid_cost = reformed_simulation_texas.calculate("medicaid_per_capita_cost", map_to="household", period=2026)

# Calculate total benefits for each scenario
baseline_vermont_total = [sum(x) for x in zip(baseline_vermont_per_capita_chip, baseline_vermont_aca_ptc, baseline_vermont_medicaid_cost)]
reform_vermont_total = [sum(x) for x in zip(reform_vermont_per_capita_chip, reform_vermont_aca_ptc, reform_vermont_medicaid_cost)]

baseline_texas_total = [sum(x) for x in zip(baseline_texas_per_capita_chip, baseline_texas_aca_ptc, baseline_texas_medicaid_cost)]
reform_texas_total = [sum(x) for x in zip(reform_texas_per_capita_chip, reform_texas_aca_ptc, reform_texas_medicaid_cost)]



In [86]:
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
TEAL_ACCENT = "#39C6C0"
DARK_GRAY = "#616161"

In [87]:
# Create Vermont graph
fig_vermont = go.Figure()

# Add baseline traces (solid lines)
fig_vermont.add_trace(go.Scatter(
    x=household_income_vermont, 
    y=baseline_vermont_per_capita_chip, 
    mode='lines', 
    name='CHIP (Baseline)', 
    line=dict(color=GRAY, width=2)
))

fig_vermont.add_trace(go.Scatter(
    x=household_income_vermont, 
    y=baseline_vermont_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Baseline)', 
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_vermont.add_trace(go.Scatter(
    x=household_income_vermont, 
    y=baseline_vermont_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Baseline)', 
    line=dict(color=TEAL_ACCENT, width=2)
))

# Add reform traces (dotted lines)
fig_vermont.add_trace(go.Scatter(
    x=household_income_vermont, 
    y=reform_vermont_per_capita_chip, 
    mode='lines', 
    name='CHIP (Reform)', 
    line=dict(color=GRAY, width=2, dash='dot')
))

fig_vermont.add_trace(go.Scatter(
    x=household_income_vermont, 
    y=reform_vermont_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Reform)', 
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

fig_vermont.add_trace(go.Scatter(
    x=household_income_vermont, 
    y=reform_vermont_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Reform)', 
    line=dict(color=TEAL_ACCENT, width=2, dash='dot')
))

# Add total lines
fig_vermont.add_trace(go.Scatter(
    x=household_income_vermont, 
    y=baseline_vermont_total, 
    mode='lines', 
    name='Total Benefits (Baseline)', 
    line=dict(color=DARK_GRAY, width=2)
))

fig_vermont.add_trace(go.Scatter(
    x=household_income_vermont, 
    y=reform_vermont_total, 
    mode='lines', 
    name='Total Benefits (Reform)', 
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Update layout
fig_vermont.update_layout(
    title='New York Household (Family of 3) - Program Benefits by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Benefit Amount',
    legend_title='Programs',
    xaxis=dict(tickformat='$,.0f', range=[0, 400000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Create Texas graph
fig_texas = go.Figure()

# Add baseline traces (solid lines)
fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=baseline_texas_per_capita_chip, 
    mode='lines', 
    name='CHIP (Baseline)', 
    line=dict(color=GRAY, width=2)
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=baseline_texas_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Baseline)', 
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=baseline_texas_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Baseline)', 
    line=dict(color=TEAL_ACCENT, width=2)
))

# Add reform traces (dotted lines)
fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=reform_texas_per_capita_chip, 
    mode='lines', 
    name='CHIP (Reform)', 
    line=dict(color=GRAY, width=2, dash='dot')
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=reform_texas_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Reform)', 
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=reform_texas_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Reform)', 
    line=dict(color=TEAL_ACCENT, width=2, dash='dot')
))

# Add total lines
fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=baseline_texas_total, 
    mode='lines', 
    name='Total Benefits (Baseline)', 
    line=dict(color=DARK_GRAY, width=2)
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=reform_texas_total, 
    mode='lines', 
    name='Total Benefits (Reform)', 
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Update layout
fig_texas.update_layout(
    title='Texas Household (Couple) - Program Benefits by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Benefit Amount',
    legend_title='Programs',
    xaxis=dict(tickformat='$,.0f', range=[0, 400000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Apply your format_fig function if it exists
# If you don't have this function defined, you can remove these lines
fig_vermont = format_fig(fig_vermont)
fig_texas = format_fig(fig_texas)

# Display the figures
fig_vermont.show()
fig_texas.show()